In [75]:
import pandas as pd
import numpy as np
from alibi_detect.cd import TabularDrift
from alibi_detect.cd import ClassifierDrift
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [4]:
df = pd.read_csv(r"H:\GitHub\NHL-Game-II\Period_1_Game_Stats_Final_ModelReady.csv")

In [5]:
df.dtypes

won                        int64
Shots                    float64
Shots_Against            float64
Goals                    float64
Goals_Against            float64
Takeaways                float64
Takeaways_Against        float64
Hits                     float64
Hits_Against             float64
Blocked Shots            float64
Blocked Shots Against    float64
Giveaways                float64
Giveaways_Against        float64
Missed Shots             float64
Missed Shots_Against     float64
Penalities               float64
Penalities_Against       float64
#Won Faceoffs            float64
#Lost Faceoffs           float64
HoA_away                   int64
HoA_home                   int64
dtype: object

In [14]:
df.head()

,won,Shots,Shots_Against,Goals,Goals_Against,Takeaways,Takeaways_Against,Hits,Hits_Against,Blocked Shots,...,Giveaways,Giveaways_Against,Missed Shots,Missed Shots_Against,Penalities,Penalities_Against,#Won Faceoffs,#Lost Faceoffs,HoA_away,HoA_home
0,0,8.0,8.0,0.0,3.0,1.0,3.0,14.0,5.0,3.0,...,6.0,7.0,4.0,0.0,1.0,1.0,10.0,12.0,1,0
1,1,8.0,8.0,3.0,0.0,3.0,1.0,5.0,14.0,3.0,...,7.0,6.0,0.0,4.0,1.0,1.0,12.0,10.0,0,1
2,1,11.0,12.0,0.0,1.0,0.0,2.0,4.0,4.0,6.0,...,2.0,0.0,6.0,3.0,1.0,3.0,11.0,9.0,1,0
3,0,12.0,11.0,1.0,0.0,2.0,0.0,4.0,4.0,8.0,...,0.0,2.0,3.0,6.0,3.0,1.0,9.0,11.0,0,1
4,1,9.0,8.0,0.0,1.0,3.0,2.0,4.0,5.0,7.0,...,7.0,5.0,3.0,8.0,2.0,1.0,7.0,11.0,1,0


In [24]:
#Drop the label column
df = df.drop("won",axis=1)

In [41]:
#Get the feature names
feature_names = df.columns

In [69]:
#Split dataset into two
#Reference game contains all data minus last 20 games
X_ref = np.array(df.iloc[:-30,:])
#Test set has last 20 games
X_new = np.array(df.iloc[-30:,:])

In [70]:
#TabularDrift: K-S tests for num features, Chi-squared for categorical
cd = TabularDrift(p_val=0.05, x_ref = X_ref)

In [71]:
#Predict general drift
preds = cd.predict(X_new, return_p_val=True, return_distance = True)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds['data']['is_drift']]))

Drift? Yes!


In [72]:
#Predict the drift for each feature
fpreds = cd.predict(X_new, drift_type="feature", return_p_val=True, return_distance=True)

In [73]:
#Get drift per feature
for f in range(cd.n_features):
    stat = 'K-S'
    fname = feature_names[f]
    is_drift = fpreds['data']['is_drift'][f]
    stat_val, p_val = fpreds['data']['distance'][f], fpreds['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

Shots -- Drift? Yes! -- K-S 0.561 -- p-value 0.000
Shots_Against -- Drift? Yes! -- K-S 0.561 -- p-value 0.000
Goals -- Drift? Yes! -- K-S 0.292 -- p-value 0.009
Goals_Against -- Drift? Yes! -- K-S 0.292 -- p-value 0.009
Takeaways -- Drift? Yes! -- K-S 0.593 -- p-value 0.000
Takeaways_Against -- Drift? Yes! -- K-S 0.660 -- p-value 0.000
Hits -- Drift? Yes! -- K-S 0.902 -- p-value 0.000
Hits_Against -- Drift? Yes! -- K-S 0.903 -- p-value 0.000
Blocked Shots -- Drift? Yes! -- K-S 0.585 -- p-value 0.000
Blocked Shots Against -- Drift? Yes! -- K-S 0.584 -- p-value 0.000
Giveaways -- Drift? Yes! -- K-S 0.480 -- p-value 0.000
Giveaways_Against -- Drift? Yes! -- K-S 0.414 -- p-value 0.000
Missed Shots -- Drift? Yes! -- K-S 0.452 -- p-value 0.000
Missed Shots_Against -- Drift? Yes! -- K-S 0.518 -- p-value 0.000
Penalities -- Drift? No! -- K-S 0.216 -- p-value 0.104
Penalities_Against -- Drift? No! -- K-S 0.215 -- p-value 0.106
#Won Faceoffs -- Drift? Yes! -- K-S 0.675 -- p-value 0.000
#Lost Fac

In [ ]:
#def plot_numerical():
    